In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly_express as px
import kaleido

In [2]:
connection = sqlite3.connect("/Users/beans/Desktop/Mod-3-Project/data/themepark.db")
cursor = connection.cursor()

In [9]:
category = """
SELECT da.attraction_name, da.category, 
       ROUND(AVG(fre.wait_minutes),2) as average_wait,
       ROUND(AVG(fre.satisfaction_rating),2) as average_rating
FROM fact_ride_events fre
JOIN dim_attraction da ON fre.attraction_id = da.attraction_id
WHERE fre.wait_minutes IS NOT NULL
GROUP BY da.category
"""

cursor.execute(category)

In [10]:
catdf = pd.read_sql_query(category,connection)

catdf



,attraction_name,category,average_wait,average_rating
0,Tiny Trucks,Kids,37.07,3.36
1,Space Theater,Show,40.44,2.78
2,Dragon Drop,Thrill,48.79,3.16
3,Pirate Splash!,Water,49.12,2.72


In [11]:
catbar = px.bar(catdf, 'category', 'average_wait', text='average_wait', title = 'Average Wait per Ride Category', width =450 ,height =500)
catbar.update_layout(xaxis_title='Ride Category', yaxis_title='Average Wait in Minutes', font=dict(size=12))
catbar.update_traces(marker_color = ['ForestGreen', 'RoyalBlue', 'RoyalBlue', 'FireBrick'],textposition='outside')
catbar

In [19]:
catbar.write_image("waitpercat.png")

In [13]:
waterfilter = """
SELECT da.attraction_name, da.category, fre.satisfaction_rating,
       fre.wait_minutes as wait,
       satisfaction_rating ating
FROM fact_ride_events fre
JOIN dim_attraction da ON fre.attraction_id = da.attraction_id
WHERE (fre.wait_minutes IS NOT NULL) AND (da.category IS 'Water')
"""

cursor.execute(waterfilter)

In [24]:
waterscat  = px.bar(catdf, 'category','average_rating', text = 'average_rating', title = 'Average Rating per Ride ', width = 450, height=500)
waterscat.update_layout(xaxis_title='Ride Category', yaxis_title='Average Rating', font=dict(size=12))
waterscat.update_traces(marker_color = ['ForestGreen', 'RoyalBlue', 'RoyalBlue', 'FireBrick'],textposition='outside')


In [ ]:
waterscat.write_image("ratepercat.png")

In [16]:
promo = """
WITH promopurchase AS (
  SELECT *
  FROM fact_purchases fp
  JOIN fact_visits fv ON fv.visit_id = fp.visit_id
)
 SELECT promotion_code,COUNT(*) as count_of_purchases
 FROM promopurchase
 WHERE promotion_code IS NOT NULL
 GROUP BY promotion_code
"""

cursor.execute(promo)

In [17]:
promodf = pd.read_sql_query(promo,connection)
promodf

,promotion_code,count_of_purchases
0,,4
1,SUMMER25,39
2,VIPDAY,11


In [25]:
#pie chart for percentages of purchase

promotionpurch = px.pie(promodf,names=['No Promotion','SUMMER25','VIPDAY'],values='count_of_purchases',title='Purchase Percentage by Promotional Offer',height=400, width=500)
promotionpurch.update_layout(font=dict(size=15))
promotionpurch.update_traces(marker_colors = ['FireBrick', 'ForestGreen', 'RoyalBlue'],textposition='outside')


In [ ]:
promotionpurch.write_image("piepercent.png")